In [ ]:
import pandas as pd
import os
os.chdir('C:/Users/928996/Desktop/Capstone Project/datasets')

df = pd.read_excel("population (1).xlsx", sheet_name = "Data")
df

Dropping 'null' and 'nan' values

In [ ]:
df.dropna(inplace = True)
df

Dropping range of rows that are not going to be used for analysing

In [ ]:
df.drop(range(22,238), inplace=True)
df.reset_index(drop=True, inplace=True)
df

Renaming column 'When' to 'Date'

In [ ]:
df.rename(columns={'When':'Date','Alberta':'ALBERTA', 'BritishColumbia':'BRITISH_COLUMBIA', 'Manitoba':'MANITOBA','NewBrunswick':'NEW_BRUNSWICK','NewfoundlandAndLabrador':'NEWFOUNDLAND_AND_LABRADOR', 'NovaScotia':'NOVA_SCOTIA','Ontario': 'ONTARIO', 'PrinceEdwardIsland': 'PRINCE_EDWARD_ISLAND','Quebec':'QUEBEC', 'Saskatchewan':'SASKATCHEWAN'},inplace = True)
df

Reshaping the DataFrame by combining miltiple columns in one column. 

In [ ]:
df3 = df.melt(id_vars = ['Date'],
        var_name = 'Location', 
         value_name = 'Population' )
df3

In [ ]:
df3 = df3.drop(df3[df3['Location'] == 'Canada'].index)
df3

In [ ]:
import pandas as pd
import os
os.chdir('C:/Users/928996/Desktop/Capstone Project/datasets')

df2 = pd.read_csv("population-by-region-1946-2022.csv")
df2

Checking for duplicates

In [ ]:
print(df2.duplicated())


Dropping unnecessesary columns. 

In [ ]:
df2 = df2.drop(['DGUID', 'VECTOR', 'COORDINATE'], axis=1)
df2

Renaiming columns. 

In [ ]:
df2.rename(columns={'REF_DATE':'Date', 'GEO':'Location', 'Population estimate': 'Population'},inplace = True)
df2

Renaiming existing values in a row. 

In [ ]:
row_names = {'Alberta':'ALBERTA', 'British Columbia':'BRITISH_COLUMBIA', 'Manitoba':'MANITOBA','New Brunswick':'NEW_BRUNSWICK','Newfoundland and Labrador':'NEWFOUNDLAND_AND_LABRADOR', 'Nova Scotia':'NOVA_SCOTIA','Ontario': 'ONTARIO', 'Prince Edward Island': 'PRINCE_EDWARD_ISLAND','Quebec':'QUEBEC', 'Saskatchewan':'SASKATCHEWAN'}
newdf2 = df2.rename(index = row_names)
newdf2


In [ ]:
df2=df2.replace({'Location':{'Alberta':'ALBERTA','British Columbia':'BRITISH_COLUMBIA','Manitoba':'MANITOBA','New Brunswick':'NEW_BRUNSWICK','Newfoundland and Labrador':'NEWFOUNDLAND_AND_LABRADOR', 'Nova Scotia':'NOVA_SCOTIA','Ontario': 'ONTARIO','Prince Edward Island': 'PRINCE_EDWARD_ISLAND','Quebec':'QUEBEC','Saskatchewan':'SASKATCHEWAN'}})
df2

Dropping values from 'Location' column. 

In [ ]:
df2 = df2.drop(df2[df2['Location'] == 'Canada'].index)
df2

In [ ]:
df2 = df2.drop(df2[(df2['Location'] == 'Yukon')].index)
df2

In [ ]:
df2 = df2.drop(df2[(df2['Location'] == 'Nunavut')].index)
df2

In [ ]:
df2 = df2.drop(df2[(df2['Location'] == 'Northwest Territories')].index)
df2

Rearranging rows. 

In [ ]:
df2.reset_index(drop=True, inplace=True)
df2

Looking for specific date in order to drop unnecessary values prior that date

In [ ]:
df2.loc[df2['Date'] == 'Oct-15']


In [ ]:
df2.drop(range(0,2731), inplace=True)
df2.reset_index(drop=True, inplace=True)
df2

Checking the number of the rows before and after the changes. 

In [ ]:
df3.shape

In [ ]:
df2.shape

Merging two DataFrames. 

In [ ]:
newdf = pd.concat([df3, df2])
newdf


Creating visual. 

In [ ]:
import matplotlib as plt

In [ ]:
import matplotlib
matplotlib.__version__

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import numpy as np

In [ ]:
x = newdf['Location']
y = newdf['Population'] 
plt.xlabel('Location')
plt.ylabel('Population')
plt.barh(x,y)


In [ ]:
import pyodbc

cnxn = pyodbc.connect(
    r'Driver=SQL Server;'
    r'Server=ADT230771;'
    r'Database=DWQueue;'
    r'Trusted_Connection=yes;'
    )
a=newdf['Location'].unique()
print(a)

cursor = cnxn.cursor()
for row in newdf.itertuples():
    cursor.execute('select ProvinceID from Province where Name = ?',row[2])
    prov = cursor.fetchone()[0]
    sql = "INSERT INTO Population (Date,Province,population) VALUES (?,?,?)"
    try:
        cursor.execute(sql,row[1],prov,row[3])
        cursor.commit()
    except:
        pass
cursor.close()
